In [3]:
import pandas as pd
import xlrd
import mip
import networkx as nx
import datetime

# definizioni costanti fittizzie

COUNTED_PASSENGERS = 0
EXTRA_PASSENGERS = 1
INITIAL_STOP = 2
FINAL_STOP = 3
INITIAL_TIME = 4
FINAL_TIME = 5
K_AUTOBUS = 33

# lettura files

Tratte = pd.read_excel('file_di_dati.xlsx', sheet_name="porzioni di viaggio")
t_v = pd.read_excel('file_di_dati.xlsx', sheet_name="tempi di percorrenza a vuoto")

#print(Tratte)
'print(tempi_vuoto)'

print('-----------------------------------------------------------')

'''
Tratte = Tratte.to_numpy()
tempi_vuoto = tempi_vuoto.to_numpy()

print(Tratte)
#print(tempi_vuoto)
'''

# metto tutte le colonne in un array

counted_passengers = pd.read_excel("file_di_dati.xlsx",'porzioni di viaggio', usecols='A', skiprows=[0]) 
counted_passengers = counted_passengers.values.tolist()

extra_passengers = pd.read_excel("file_di_dati.xlsx",'porzioni di viaggio', usecols='B', skiprows=[0])
extra_passengers = extra_passengers.values.tolist() 
n_extra_passengers = len(extra_passengers)
n_tratte = n_extra_passengers

initial_stop = pd.read_excel("file_di_dati.xlsx",'porzioni di viaggio', usecols='C', skiprows=[0]) 
initial_stop = initial_stop.values.tolist()


final_stop = pd.read_excel("file_di_dati.xlsx",'porzioni di viaggio', usecols='D', skiprows=[0]) 
final_stop = final_stop.values.tolist()

initial_time = pd.read_excel("file_di_dati.xlsx",'porzioni di viaggio', usecols='E', skiprows=[0]) 
initial_time = initial_time.values.tolist()

final_time = pd.read_excel("file_di_dati.xlsx",'porzioni di viaggio', usecols='F', skiprows=[0]) 
final_time = final_time.values.tolist()

#print(final_time[0][0]-initial_time[0][0])
#print(initial_stop[0][0])

# funzione per convertie i tempi

epoch = datetime.datetime.utcfromtimestamp(0)

def unix_time_sec(dt):
    return (dt - epoch).total_seconds()



-----------------------------------------------------------


In [4]:
# uniformo i nomi tra tempi di percorrenza e tempi a vuoto

for i in range(n_tratte):

    initial_stop[i][0].strip()
    final_stop[i][0].strip()

    if initial_stop[i][0] == 'stazione fs':
         initial_stop[i][0] = 'stazione'

    if initial_stop[i][0] == 'p. medaglie d’oro':
        initial_stop[i][0] = 'medaglie d’oro'

    if initial_stop[i][0] == 'piazzale medaglie d’oro':
        initial_stop[i][0] = 'medaglie d’oro'

    if initial_stop[i][0] == 'piazzale dante':
        initial_stop[i][0] = 'p. dante'

    if final_stop[i][0] == 'stazione fs':
         final_stop[i][0] = 'stazione'

    if final_stop[i][0] == 'p. medaglie d’oro':
        final_stop[i][0] = 'medaglie d’oro'

    if final_stop[i][0] == 'piazzale medaglie d’oro':
        final_stop[i][0] = 'medaglie d’oro'

    if final_stop[i][0] == 'piazzale dante':
        final_stop[i][0] = 'p. dante'

for i in range(1,n_tratte):
    initial_stop[i][0] = initial_stop[i][0].strip()
    final_stop[i][0] = final_stop[i][0].strip()



In [5]:

# qua bisogna sistemare i nomi di tempi a vuoto
#questo for rende tutti i nomi omogenei a quelli delle righa zeresima cambiando la col zeresima

tempi_vuoto = (t_v.values.tolist())
n_luoghi = len(tempi_vuoto[0][:])-1

for i in range(n_luoghi):
    if tempi_vuoto[i][0]!=tempi_vuoto[0][i]:
        print(tempi_vuoto[i][0])
        tempi_vuoto[i][0]=tempi_vuoto[0][i]

#ribalto la matrice per potere scrivere tratte_vuoto[i][j]
for i in range(n_luoghi+1):
    for j in range(n_luoghi+1):
        if i>j:
            tempi_vuoto[j][i]=tempi_vuoto[i][j]
        if i == j:
            tempi_vuoto[i][j]=0

luoghi = tempi_vuoto[0][:]
luoghi.pop(0)

print(luoghi)
#print(n_luoghi)
#print(tempi_vuoto)

luoghi_to_indici = {luoghi[i]:i for i in range(n_luoghi)}


# usiamo un dizionario in cui ogni nome corrisponde a un indice

#print(luoghi_to_indici[luoghi[2]])




nan
['stazione', 'kennedy', 'barriera', 'canapa', 'ospedale', 'medaglie d’oro', 'p. dante', 'pontegradella itis', 'barlaam', 'cavour giardini', 'santa maria maddalena', 'autostazione', 'corti di medoro', 'giovecca montebello', 'barco', 'malborghetto']


In [6]:
# i nodi sono per ogni tratta 2
# in particolare uno è la partenza e uno l' arrivo
# definisco tutti i nodi, li collgo in base alla tratta 
# definisco anche il nodo pozzo e il nodo sorgente

s = 0
t = n_tratte+1

nodi_partenze = []

for i in range(1,n_tratte+1):
    nodi_partenze.append(i)

nodi_arrivi = []

for i in range(1,n_tratte+1):
    nodi_arrivi.append(i)


print(nodi_arrivi)



[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156]


In [11]:
A = []

print(luoghi_to_indici[final_stop[1][0]])

# arco s
s = 0
for j in range (n_tratte):
    A.append((s,j))
# archi compatibilità

for i in range(n_tratte):
    indice_luogo_corrente = luoghi_to_indici[final_stop[i][0]]
    for j in range(n_tratte):
        indice_luogo_desiderato = luoghi_to_indici[initial_stop[j][0]]
        if i!= j and initial_time[j][0] <= final_time[i][0] + tempi_vuoto[indice_luogo_corrente+1][indice_luogo_desiderato+1]:
            A.append((i+1,j+1))

# archi bilancio
#arco t
t = n_tratte+2
for i in range(n_tratte):
    A.append((i,t))

print(A)

30), (145, 31), (145, 32), (145, 33), (145, 34), (145, 35), (145, 36), (145, 37), (145, 38), (145, 39), (145, 40), (145, 41), (145, 42), (145, 43), (145, 44), (145, 45), (145, 46), (145, 47), (145, 48), (145, 49), (145, 50), (145, 51), (145, 52), (145, 53), (145, 54), (145, 55), (145, 56), (145, 57), (145, 58), (145, 59), (145, 60), (145, 61), (145, 62), (145, 63), (145, 64), (145, 65), (145, 66), (145, 67), (145, 68), (145, 69), (145, 70), (145, 71), (145, 72), (145, 73), (145, 74), (145, 75), (145, 76), (145, 77), (145, 78), (145, 79), (145, 80), (145, 81), (145, 82), (145, 83), (145, 84), (145, 85), (145, 86), (145, 87), (145, 88), (145, 89), (145, 90), (145, 91), (145, 92), (145, 93), (145, 94), (145, 95), (145, 96), (145, 97), (145, 98), (145, 99), (145, 100), (145, 101), (145, 102), (145, 103), (145, 104), (145, 105), (145, 106), (145, 107), (145, 108), (145, 109), (145, 110), (145, 111), (145, 112), (145, 113), (145, 114), (145, 115), (145, 116), (145, 117), (145, 118), (145, 11